In [2]:
import pandas as pd
import os
from nltk.stem.wordnet import WordNetLemmatizer
lmtzr = WordNetLemmatizer()
from difflib import SequenceMatcher
import operator
import nltk
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.util import ngrams

[nltk_data] Downloading package wordnet to /home/ubuntu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
#Pull ingredient list + classification
# df = pd.read_csv('recipe_app/data/recipe_ingredients_new.csv',index_col=False, encoding='latin1')
df_new = pd.read_csv('recipe_app/data/final_recipe_list_categorized.csv',index_col=0)

In [4]:
df_new['length'] = df_new['ingredient'].apply(lambda x: len(x))

In [5]:
df_new = df_new[df_new['length'] > 3]

In [6]:
df_new[df_new['ingredient'] == 'baking powder']

,ingredient,category,length
429,baking powder,miscellaneous,13
8530,baking powder,miscellaneous,13


In [7]:
df_new['category'].unique()

array(['dairy-milk', 'meat-lamb', 'meat-beef', 'liquor', 'spices',
       'vegetable-mushrooms', 'vegetable-squash', 'fruit-candy',
       'meat-sausage', 'legumes', 'fruit', 'vinegars', 'fruit-tropical',
       'condiments', 'legumes-nuts', 'flours', 'fruit-melons',
       'vegetable-greens', 'thickeners', 'dairy-cheese',
       'vegetable-olives', 'vegetable-tuber', 'vegetable-peppers', 'wine',
       'salt', 'seafood-fish', 'seafood-shellfish', 'vegetable-sprouts',
       'vegetable-sea', 'fruit-berries', 'meat-other',
       'vegetable-avocados', 'extracts', 'dairy-non', 'sweetener',
       'meat-coldcuts', 'miscellaneous', 'cookies', 'meat-miscellaneous',
       'beer', 'fruit-pome', 'fruit-dry', 'herbs', 'meat-bacon',
       'seeds-rice', 'seafood-caviar', 'chocolate', 'vegetable-eggplant',
       'cakes', 'dairy-butters', 'vegetable-stalk', 'flowers',
       'fruit-juice', 'fruit-stone', 'fruit-preserves',
       'fruits-preserves', 'vegetable-onions', 'dairy-cultured', 'bread',

In [8]:
df = pd.DataFrame(df_new['category'].unique(), columns=['category'])

In [9]:
df.to_csv('category.csv')

## Test area with three recipes

In [10]:
# Need list of metrics for standardizing

#you buy oz or mL
#pounds or grams

metrics = ['tablespoon', 'tablespoons', 'teaspoon', 'teaspoons', 'tbsp', 'tbsps', 'tsp', \
           'tsps', 'cup', 'cups', 'ounce', 'ounces', 'oz', 'ozs'\
           'quart', 'quarts', 'qt', 'qts', 'pt', 'pts', 'pint', 'pints', 'gallon', 'gallons', \
           'gal', 'gals', 'pound', 'pounds', 'lb', 'lbs', 'g', 'gs',\
           'gram', 'grams', 'kilogram', 'kilograms', 'kg', 'liter', 'liters', 'L', 'millileter', \
           'mL', 'millileters']

In [35]:
recipe_ingredients = ['ounce country-style bread, crusts removed', 'tablespoons olive oil, divided', \
                      'scallions', 'salt, freshly ground pepper', 'tablespoons fresh lemon juice', \
                      'tablespoons unseasoned rice vinegar', 'tablespoons Dijon mustard', \
                      'tablespoons mayonnaise, preferably Hellmann’s', \
                      'small rotisserie chicken, meat pulled from bone', 'radishes, trimmed, cut into wedges', \
                      'head of Bibb lettuce, leaves separated', 'avocado, sliced, divided', 'pound ground lamb', \
                      'cup yellow or red onion, ﬁnely chopped', 'cloves garlic, minced', 'teaspoons fresh ginger, ﬁnely grated', \
                      'teaspoon garam masala', 'teaspoon curry powder', 'teaspoon paprika', 'Salt', 'Pepper', 'ramps, just the greens, minced', 
                      'tablespoons extra-virgin olive oil', 'teaspoons lemon juice', 'cup yogurt', 'large ﬂour tortillas or wraps', \
                      'Persian cucumbers, sliced very thin on a mandoline', 'red onion, thinly sliced', \
                      'Fresh cilantro sprigs, about 1/4 bunch', 'Greens or microgreens', 'Nonstick vegetable oil spray', \
                      'cups all-purpose flour', 'cup sugar', 'teaspoons baking powder', 'teaspoon kosher salt', \
                      'large eggs', 'cups ricotta', 'teaspoon vanilla extract', 'cup (1 stick) unsalted butter, melted', \
                      'cup frozen raspberries or blackberries, divided', 'ounces linguine', 'Kosher salt', \
                      'tablespoons unsalted butter, divided', 'tablespoons olive oil, divided', 'cup minced shallots', \
                      'teaspoon minced garlic', 'Fresno chiles, red jalapeños, or red Thai chiles, seeded, sliced into thin rounds', \
                      'tablespoons (or more) fresh lemon juice, divided', 'teaspoons (packed) finely grated lemon zest, divided', \
                      'Freshly ground black pepper', 'ounces cooked, shelled Dungeness crab, king crab, or jumbo lump crabmeat, picked over for shells', \
                      'cup fresh mint leaves, gently torn, divided']

# recipe_ingredients = ['teaspoons baking powder']

In [36]:
target_list = pd.DataFrame(recipe_ingredients, columns=['recipe_ingredients'])

In [37]:
target_list['original_recipe'] = target_list['recipe_ingredients']

In [38]:
#Find words in metrics list for recipe list generation

def strip_measurements(x, metrics):
    found_metrics = ''
    keep_words = ''
    for word in x.split(' '):
        if word in metrics:
            found_metrics = found_metrics + ' ' + word
        else: 
            keep_words = keep_words + ' ' + word
    return keep_words + '|' + found_metrics

target_list['recipe_ingredients'] = target_list['recipe_ingredients'].apply(lambda x: strip_measurements(x, metrics))

In [39]:
def add_metrics(x):
    x = x.split('|')
    return x[1]

def remove_metrics(x):
    x = x.split('|')
    return x[0]

target_list['metric'] = target_list['recipe_ingredients'].apply(lambda x: add_metrics(x))
target_list['recipe_ingredients'] = target_list['recipe_ingredients'].apply(lambda x: remove_metrics(x))

In [40]:
# import re
# def reg(x):
#     x = x.replace(',', ' a')
#     x = x.replace('-', ' ')
#     return x

# target_list['recipe_ingredients'] = target_list['recipe_ingredients'].apply(lambda x: reg(x))

In [41]:
def pos(x):
    tokens = nltk.word_tokenize(x)
    tagged_list = nltk.pos_tag(tokens)
    new_string = ''
    for tup in tagged_list:
        if tup[1] in ['NN', 'JJ', 'NNS', 'NNP', 'VBP', 'VBG']:
            new_string += tup[0] + ' '
        else:
            pass
    return new_string.lower()
target_list['recipe_ingredients'] = target_list['recipe_ingredients'].apply(lambda x: pos(x))

In [42]:
# new_recipe_ingredients = []
# substitutes = []
# for recipe in recipe_ingredients:
#     if type(recipe) == list:
#         temp_list = []
#         for element in recipe:
#             tokens = nltk.word_tokenize(element[0])
#             tagged_list = nltk.pos_tag(tokens)
#             new_string = ''
#             for tup in tagged_list:
#                 if tup[1] in ['NN', 'JJ', 'NNS', 'NNP']:
#                     new_string += tup[0] + ' '
#             else:
#                 pass
#             if len(new_string) > 0:
#                 temp_list.append(new_string)
#         new_recipe_ingredients.append(temp_list[0])
#         try:
#             substitutes.append(temp_list[1:])
#         except:
#             pass
#     else:
#         tokens = nltk.word_tokenize(recipe)
#         tagged_list = nltk.pos_tag(tokens)
#         new_string = ''
#         for tup in tagged_list:
#             if tup[1] in ['NN', 'JJ', 'NNS', 'NNP']:
#                 new_string += tup[0] + ' '
#             else:
#                 pass
#         new_recipe_ingredients.append(new_string)

In [43]:
# import nltk
# import pandas as pd
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
# new_recipe_ingredients = list(target_list['recipe_ingredients'])
# for recipe in recipe_ingredients:
#     if ' or ' in recipe:
#         final = [y.replace(',', '').split(' or ') for y in recipe.split(',')]
#         new_recipe_ingredients.append(final)
#     else:
#         new_recipe_ingredients.append(recipe)
# target_list['recipe_ingredients'] = new_recipe_ingredients

In [44]:
def try_lemmatize(x):
    try:
        return_string = lmtzr.lemmatize(x).lower().encode('ascii')
    except:
        return_string = x.lower()
    return return_string

def lemmatizer(x):
    lmtzr = WordNetLemmatizer()
    return ' '.join([try_lemmatize(y) for y in x.split(' ')])
    
target_list['recipe_ingredients'] = target_list['recipe_ingredients'].apply(lambda x: lemmatizer(x))


#create list of ngrms turned into individual strings
#['I', 'am', 'I am']

def get_ngrams(text):
    final = []
    length = text.split(' ')
    length = len(length)
    for n in range(1, (length+1)):
        n_grams = ngrams(word_tokenize(text), n)
        final.append([ ' '.join(grams) for grams in n_grams])
    return [item for sublist in final for item in sublist]

#Find longest matching string in 'ingredient' field in database

def standardize_ingredients(row, ingredients):
    possible_standard = []
    for gram in get_ngrams(row['recipe_ingredients']):
        try:
            possible_standard.append(ingredients[ingredients['ingredient'] == gram]['ingredient'].iloc[0])
        except:
            pass
    if len(possible_standard) == 0:
        return row['recipe_ingredients'] + '|' + 'not categorized'
    else:
        return max(possible_standard, key=len) + '|' + ingredients['category'][ingredients['ingredient'] == max(possible_standard, key=len)].iloc[0]


# def standardize_ingredients(row, ingredients):
#     possible_standard = []
#     for ingredient in get_ngrams(row['recipe_ingredients']):
#         if ingredient in ingredients:
#             possible_standard.append(ingredient)
#     if len(possible_standard) == 0:
#         return row['recipe_ingredients'] + '|' + 'not categorized'
#     else:
#         return max(possible_standard, key=len) + '|' + df_new['category'][df_new['ingredient'] == max(possible_standard, key=len)].iloc[0]

target_list['recipe_ingredients'] = target_list.apply(lambda row: standardize_ingredients(row, df_new), axis=1)
target_list['category'] = target_list['recipe_ingredients'].apply(lambda x: add_metrics(x))
target_list['recipe_ingredients'] = target_list['recipe_ingredients'].apply(lambda x: remove_metrics(x))

In [45]:
target_list = target_list.groupby(['recipe_ingredients', 'category']).agg(lambda x: x.tolist()).sort_values(['category']).reset_index()

In [46]:
target_list

,recipe_ingredients,category,original_recipe,metric
0,bread,bread,"[ounce country-style bread, crusts removed]",[ ounce]
1,tortilla wrap,bread,[large ﬂour tortillas or wraps],[]
2,mayonnaise,condiments,"[tablespoons mayonnaise, preferably Hellmann’s]",[ tablespoons]
3,dijon mustard,condiments,[tablespoons Dijon mustard],[ tablespoons]
4,unsalted butter,dairy-butters,"[tablespoons unsalted butter, divided]",[ tablespoons]
5,butter,dairy-butters,"[cup (1 stick) unsalted butter, melted]",[ cup]
6,ricotta,dairy-cheese,[cups ricotta],[ cups]
7,yogurt,dairy-cultured,[cup yogurt],[ cup]
8,vanilla extract,extracts,[teaspoon vanilla extract],[ teaspoon]
9,linguine,flours,[ounces linguine],[ ounces]


## Categorize ingredients

In [107]:
#Check ingredient and string match with already classified ingredients to find close match (to then classify)

def seq_matcher(string_one, df):
    x = string_one
    for index, value in df.iterrows():
        s = SequenceMatcher(None, string_one, value['ingredient'])
        if s.ratio() > 0.75:
            x = value['ingredient']
        else:
            pass
    print string_one, x
    return x

In [7]:
#Taking the columns I need

df = df[['ingredients', 'category']]
df_new = df_new[['ingredient', 'category']]

In [30]:
#unicode to ascii

import re
def reg(x):
    try:
        x = " ".join(re.findall("[a-zA-Z]+", x))
    except:
        x = x
    return x
df_new['ingredient'] = df_new['ingredient'].apply(lambda x: reg(x))
df_new['ingredient'] = [str(line).decode('utf-8').strip() for line in df_new['ingredient']]

In [15]:
#Lemmatize ingredients strings and lowercase to merge

df['ingredients'] = df['ingredients'].apply(lambda x: ', '.join([lmtzr.lemmatize(y).lower() for y in x.split(' ')]).replace(',', ''))
df_new['ingredient'] = df_new['ingredient'].apply(lambda x: ', '.join([lmtzr.lemmatize(y).lower() for y in str(x).split(' ')]).replace(',', ''))

In [34]:
#Merge on ingredients and drop duplicates

merged_df = df_new.merge(df, left_on='ingredient', right_on='ingredients', how='outer').drop_duplicates()

In [35]:
#Check which ingredients were merged and seperate still not classified ingredients

not_classified = merged_df[merged_df['category_x'] != merged_df['category_x']]
alreay_classified = merged_df[merged_df['category_x'] == merged_df['category_x']]

## Scraping data from Food Dictionary

In [ ]:
# #Scraper

# import requests
# from bs4 import BeautifulSoup
# import pandas as pd
# import urllib2
# import unicodedata

# class food_dictionary:
#     def __init__(self):
#         self.df = pd.DataFrame(columns = ['ingredient', 'category'])

#     def turn_to_df(self, category, url):
#         quote_page = url
#         page = requests.get(quote_page)
#         soup = BeautifulSoup(unicodedata.normalize('NFKD', page.text), 'html.parser')
#         x = soup.find_all("b")
#         recipe_list = []
#         for y in x:
#             recipe_list.append(y.text)
#         recipe_list = self.clean_list(recipe_list)
#         df = pd.DataFrame(recipe_list, columns=['ingredient'])
#         df['category'] = category
#         self.df = self.df.append(df)
        
#     def clean_list(self, recipe_list):
#         new_list = []
#         for x in recipe_list:
#             x = x.split('Notes:', 1)[0]
#             x = x.split('See ', 1)[0]
#             if ':' in x:
#                 pass
#             else:
#                 if '=' in x:
#                     for equivalent in x.split('='):
#                         if equivalent == u'':
#                             pass
#                         else:
#                             new_list.append(equivalent.replace('\n', '').replace(u'\xa0', u'').strip().replace('  ', ' '))
#                 else:
#                     if x == u'':
#                         pass
#                     else:
#                         new_list.append(x.replace('\n', ' ').replace(u'\xa0', u'').strip().replace('  ', ' '))
#         return new_list[1:]

In [ ]:
# #List of urls and classification of ingredients on page

# scrape_list = [('vegetable-tuber', 'http://www.foodsubs.com/Tubers.html'), \
#                ('vegetable-root', 'http://www.foodsubs.com/Roots.html'), \
#                ('vegetable-potatoes','http://www.foodsubs.com/Potatoes.html'), \
#                ('vegetable-sweet potatoes','http://www.foodsubs.com/Sweetpotatoes.html'), \
#                ('vegetable-stalk', 'http://www.foodsubs.com/Stalk.html'), \
#                ('vegetable-onions', 'http://www.foodsubs.com/Onionsgreen.html'), \
#                ('vegetable-onions', 'http://www.foodsubs.com/Onionsdry.html'), \
#                ('vegetable-garlic', 'http://www.foodsubs.com/Garlic.html'), \
#                ('vegetable-ginger', 'http://www.foodsubs.com/Ginger.html'), \
#                ('vegetable-cabbage', 'http://www.foodsubs.com/Cabbage.html'), \
#                ('vegetable-greens', 'http://www.foodsubs.com/Greensld.html'), \
#                ('vegetable-greens', 'http://www.foodsubs.com/Greenckg.html'), \
#                ('vegetable-flower', 'http://www.foodsubs.com/Vegiesinflor.html'), \
#                ('vegetable-bean', 'http://www.foodsubs.com/Snapbean.html'), \
#                ('vegetable-pods', 'http://www.foodsubs.com/Pods.html'), \
#                ('vegetable-peas', 'http://www.foodsubs.com/Shellpeas.html'), \
#                ('vegetable-ben', 'http://www.foodsubs.com/Shellbeans.html'), \
#                ('vegetable-mushrooms', 'http://www.foodsubs.com/Mushroom.html'), \
#                ('vegetable-fruits', 'http://www.foodsubs.com/Fruitvegies.html'), \
#                ('vegetable-tomatoes', 'http://www.foodsubs.com/Tomtom.html'), \
#                ('vegetable-eggplant', 'http://www.foodsubs.com/Eggplants.html'), \
#                ('vegetable-squash', 'http://www.foodsubs.com/Squash.html'), \
#                ('vegetable-squash', 'http://www.foodsubs.com/Squashsum.html'), \
#                ('vegetable-squash', 'http://www.foodsubs.com/Squashasian.html'), \
#                ('vegetable-cucumber', 'http://www.foodsubs.com/Squcuke.html'), \
#                ('vegetable-peppers', 'http://www.foodsubs.com/Peppersw.html'), \
#                ('vegetable-peppers', 'http://www.foodsubs.com/Chilefre.html'), \
#                ('vegetable-peppers', 'http://www.foodsubs.com/Peppersw.html'), \
#                ('vegetable-peppers', 'http://www.foodsubs.com/Chiledry.html'), \
#                ('vegetable-olives', 'http://www.foodsubs.com/Olivpick.html'), \
#                ('vegetable-avocados', 'http://www.foodsubs.com/Avocados.html'), \
#                ('vegetable-sea', 'http://www.foodsubs.com/Seaveg.html'), \
#                ('vegetable-sprouts', 'http://www.foodsubs.com/Sprouts.html'), \
#                ('vegetable-other', 'http://www.foodsubs.com/Vegies.html'), \
#                ('fruit-citrus', 'http://www.foodsubs.com/Fruitcit.html'), \
#                ('fruit-berries', 'http://www.foodsubs.com/Fruitber.html'), \
#                ('fruit-stone', 'http://www.foodsubs.com/Fruitsto.html'), \
#                ('fruit-tropical', 'http://www.foodsubs.com/Fruittro.html'), \
#                ('fruit-tropical', 'http://www.foodsubs.com/Fruittroex.html'), \
#                ('fruit-melons', 'http://www.foodsubs.com/Fruitmel.html'), \
#                ('fruit-dry', 'http://www.foodsubs.com/Fruitdry.html'), \
#                ('fruit-pome', 'http://www.foodsubs.com/Fruitoth.html'), \
#                ('fruit-pome', 'http://www.foodsubs.com/Apples.html'), \
#                ('fruit-pome', 'http://www.foodsubs.com/Pears.html'), \
#                ('fruit-preserves', 'http://www.foodsubs.com/Fruitpre.html'), \
#                ('fruit-candy', 'http://www.foodsubs.com/Candied.html'), \
#                ('fruit-juice', 'http://www.foodsubs.com/Juice.html'), \
#                ('dairy-milk', 'http://www.foodsubs.com/Dairyoth.html'), \
#                ('dairy-cultured', 'http://www.foodsubs.com/Cultmilk.html'), \
#                ('dairy-non', 'http://www.foodsubs.com/Nondairy.html'), \
#                ('dairy-cheese', 'http://www.foodsubs.com/Cheese.html'), \
#                ('dairy-cheese', 'http://www.foodsubs.com/Chefresh.html'), \
#                ('dairy-cheese', 'http://www.foodsubs.com/Chesoft.html'), \
#                ('dairy-cheese', 'http://www.foodsubs.com/Chessoft.html'), \
#                ('dairy-cheese', 'http://www.foodsubs.com/Chesfirm.html'), \
#                ('dairy-cheese', 'http://www.foodsubs.com/Chefirm.html'), \
#                ('dairy-cheese', 'http://www.foodsubs.com/Cheblue.html'), \
#                ('dairy-cheese', 'http://www.foodsubs.com/Cheprocessed.html'), \
#                ('dairy-cheese', 'http://www.foodsubs.com/CheeseAlt.html'), \
#                ('dairy-egg', 'http://www.foodsubs.com/Eggs.html'), \
#                ('herbs', 'http://www.foodsubs.com/HerbsUniv.html'), \
#                ('herbs', 'http://www.foodsubs.com/HerbsAfrican.html'), \
#                ('herbs', 'http://www.foodsubs.com/HerbsAmerican.html'), \
#                ('herbs', 'http://www.foodsubs.com/HerbsAsian.html'), \
#                ('herbs', 'http://www.foodsubs.com/HerbsEur.html'), \
#                ('herbs', 'http://www.foodsubs.com/HerbsHisp.html'), \
#                ('herbs', 'http://www.foodsubs.com/HerbsIndian.html'), \
#                ('herbs', 'http://www.foodsubs.com/HerbsMiddleEast.html'), \
#                ('spices', 'http://www.foodsubs.com/SpiceUniv.html'), \
#                ('spices', 'http://www.foodsubs.com/SpiceAfr.html'), \
#                ('spices', 'http://www.foodsubs.com/SpiceAsian.html'), \
#                ('spices', 'http://www.foodsubs.com/SpiceEur.html'), \
#                ('spices', 'http://www.foodsubs.com/SpiceHisp.html'), \
#                ('spices', 'http://www.foodsubs.com/SpiceInd.html'), \
#                ('spices', 'http://www.foodsubs.com/SpicemixAfr.html'), \
#                ('spices', 'http://www.foodsubs.com/SpicemixAmer.html'), \
#                ('spices', 'http://www.foodsubs.com/SpicemixAsian.html'), \
#                ('spices', 'http://www.foodsubs.com/SpicemixEur.html'), \
#                ('spices', 'http://www.foodsubs.com/SpicemixHisp.html'), \
#                ('spices', 'http://www.foodsubs.com/SpicemixInd.html'), \
#                ('spices', 'http://www.foodsubs.com/SpicemixME.html'), \
#                ('seeds', 'http://www.foodsubs.com/Seeds.html'), \
#                ('seeds-grain', 'http://www.foodsubs.com/FGGrains.html'), \
#                ('seeds-rice', 'http://www.foodsubs.com/Rice.html'), \
#                ('seeds-wheat', 'http://www.foodsubs.com/GrainWheat.html'), \
#                ('seeds-corn', 'http://www.foodsubs.com/GrainCorn.html'), \
#                ('seeds-oats', 'http://www.foodsubs.com/GrainOats.html'), \
#                ('seeds-barley', 'http://www.foodsubs.com/GrainBarley.html'), \
#                ('seeds-buckwheat', 'http://www.foodsubs.com/GrainBuckwheat.html'), \
#                ('seeds-rye', 'http://www.foodsubs.com/GrainRye.html'), \
#                ('seeds', 'http://www.foodsubs.com/GrainTrit.html'), \
#                ('seeds-spelt', 'http://www.foodsubs.com/GrainSpelt.html'), \
#                ('seeds', 'http://www.foodsubs.com/Grainoth.html'), \
#                ('legumes', 'http://www.foodsubs.com/Peas.html'), \
#                ('legumes', 'http://www.foodsubs.com/Lentils.html'), \
#                ('legumes', 'http://www.foodsubs.com/Beans.html'), \
#                ('legumes', 'http://www.foodsubs.com/Snapbean.html'), \
#                ('legumes', 'http://www.foodsubs.com/Pods.html'), \
#                ('legumes-nuts', 'http://www.foodsubs.com/Nuts.html'), \
#                ('legumes', 'http://www.foodsubs.com/Nutseed.html'), \
#                ('legumes', 'http://www.foodsubs.com/Nutmeals.html'), \
#                ('legumes', 'http://www.foodsubs.com/Soyprod.html'), \
#                ('legumes', 'http://www.foodsubs.com/BeanProducts.html'), \
#                ('extracts', 'http://www.foodsubs.com/Extracts.html'), \
#                ('salt', 'http://www.foodsubs.com/Salt.html'), \
#                ('sugars', 'http://www.foodsubs.com/Sweeten.html'), \
#                ('sweetener', 'http://www.foodsubs.com/Syrups.html'), \
#                ('chocolate', 'http://www.foodsubs.com/Chocvan.html'), \
#                ('candy', 'http://www.foodsubs.com/Candy.html'), \
#                ('condiments', 'http://www.foodsubs.com/CondimntAf.html'), \
#                ('condiments', 'http://www.foodsubs.com/CondimntAmerican.html'), \
#                ('condiments', 'http://www.foodsubs.com/CondimntAsia.html'), \
#                ('condiments', 'http://www.foodsubs.com/CondimntEur.html'), \
#                ('condiments', 'http://www.foodsubs.com/CondimntHisp.html'), \
#                ('condiments', 'http://www.foodsubs.com/CondimntInd.html'), \
#                ('condiments', 'http://www.foodsubs.com/CondimntME.html'), \
#                ('vinegars', 'http://www.foodsubs.com/Vinegars.html'), \
#                ('liquor', 'http://www.foodsubs.com/LiqueurAnise.html'), \
#                ('liquor', 'http://www.foodsubs.com/LiqueurBitters.html'), \
#                ('liquor', 'http://www.foodsubs.com/LiqueurChoc.html'), \
#                ('liquor', 'http://www.foodsubs.com/LiqueurFruit.html'), \
#                ('liquor', 'http://www.foodsubs.com/LiqueurHerbal.html'), \
#                ('liquor', 'http://www.foodsubs.com/LiqueurMisc.html'), \
#                ('liquor', 'http://www.foodsubs.com/Liqueurs.html'), \
#                ('liquor', 'http://www.foodsubs.com/Aperitif.html'), \
#                ('liquor', 'http://www.foodsubs.com/Brandy.html'), \
#                ('wine', 'http://www.foodsubs.com/WinesRed.html'), \
#                ('wine', 'http://www.foodsubs.com/WinesWhite.html'), \
#                ('wine', 'http://www.foodsubs.com/WinesBlush.html'), \
#                ('wine', 'http://www.foodsubs.com/WinesSparkling.html'), \
#                ('wine', 'http://www.foodsubs.com/WinesDessert.html'), \
#                ('wine', 'http://www.foodsubs.com/WinesFruit.html'), \
#                ('wine', 'http://www.foodsubs.com/Aperitif.html'), \
#                ('wine', 'http://www.foodsubs.com/WinesRice.html'), \
#                ('wine', 'http://www.foodsubs.com/WinesRice.html'), \
#                ('liquor', 'http://www.foodsubs.com/Liquor.html'), \
#                ('beer', 'http://www.foodsubs.com/Beer.html'), \
#                ('bitters', 'http://www.foodsubs.com/Bitters.html'), \
#                ('flours', 'http://www.foodsubs.com/Flour.html'), \
#                ('flours', 'http://www.foodsubs.com/Flournw.html'), \
#                ('flours', 'http://www.foodsubs.com/Nutmeals.html'), \
#                ('flours', 'http://www.foodsubs.com/Pasta.html'), \
#                ('flours', 'http://www.foodsubs.com/PastaRods.html'), \
#                ('flours', 'http://www.foodsubs.com/PastaRibbons.html'), \
#                ('flours', 'http://www.foodsubs.com/PastaShapes.html'), \
#                ('flours', 'http://www.foodsubs.com/PastaSoup.html'), \
#                ('flours', 'http://www.foodsubs.com/PastaTubes.html'), \
#                ('flours', 'http://www.foodsubs.com/PastaStuffed.html'), \
#                ('flours', 'http://www.foodsubs.com/Noodles.html'), \
#                ('flours', 'http://www.foodsubs.com/NoodlesWheat.html'), \
#                ('flours', 'http://www.foodsubs.com/NoodlesRice.html'), \
#                ('flours', 'http://www.foodsubs.com/NoodlesAsianOther.html'), \
#                ('flours', 'http://www.foodsubs.com/NoodlesOther.html'), \
#                ('flours', 'http://www.foodsubs.com/Dough.html'), \
#                ('thickeners', 'http://www.foodsubs.com/Thicken.html'), \
#                ('thickeners', 'http://www.foodsubs.com/ThickenStarch.html'), \
#                ('thickeners', 'http://www.foodsubs.com/ThickenGelatins.html'), \
#                ('miscellaneous', 'http://www.foodsubs.com/Wrappers.html'), \
#                ('bread', 'http://www.foodsubs.com/Bread.html'), \
#                ('cookies', 'http://www.foodsubs.com/Cookies.html'), \
#                ('cakes', 'http://www.foodsubs.com/Cakes.html'), \
#                ('crackers', 'http://www.foodsubs.com/Crackers.html'), \
#                ('bread', 'http://www.foodsubs.com/Crumbs.html'), \
#                ('bread', 'http://www.foodsubs.com/Flatbread.html'), \
#                ('meat-poultry', 'http://www.foodsubs.com/Poultry.html'), \
#                ('meat-beef', 'http://www.foodsubs.com/MeatBeefChuck.html'), \
#                ('meat-beef', 'http://www.foodsubs.com/MeatBeefRibs.html'), \
#                ('meat-beef', 'http://www.foodsubs.com/MeatBeefLoin.html'), \
#                ('meat-beef', 'http://www.foodsubs.com/MeatBeefRound.html'), \
#                ('meat-beef', 'http://www.foodsubs.com/MeatBeefB&F.html'), \
#                ('meat-beef', 'http://www.foodsubs.com/MeatBeefMisc.html'), \
#                ('meat-pork', 'http://www.foodsubs.com/MeatPorkShoulder.html'), \
#                ('meat-pork', 'http://www.foodsubs.com/MeatPorkLoin.html'), \
#                ('meat-pork', 'http://www.foodsubs.com/MeatPorkLeg.html'), \
#                ('meat-pork', 'http://www.foodsubs.com/MeatPorkSide.html'), \
#                ('meat-pork', 'http://www.foodsubs.com/MeatPorkPicnic.html'), \
#                ('meat-pork', 'http://www.foodsubs.com/MeatPorkMisc.html'), \
#                ('meat-lamb', 'http://www.foodsubs.com/MeatLambShoulder.html'), \
#                ('meat-lamb', 'http://www.foodsubs.com/MeatLambRibs.html'), \
#                ('meat-lamb', 'http://www.foodsubs.com/MeatLambLoin.html'), \
#                ('meat-lamb', 'http://www.foodsubs.com/MeatLambLeg.html'), \
#                ('meat-lamb', 'http://www.foodsubs.com/MeatLambBreast.html'), \
#                ('meat-lamb', 'http://www.foodsubs.com/MeatLambMisc.html'), \
#                ('meat-veal', 'http://www.foodsubs.com/MeatVealShoulder.html'), \
#                ('meat-veal', 'http://www.foodsubs.com/MeatVealRibs.html'), \
#                ('meat-veal', 'http://www.foodsubs.com/MeatVealLoin.html'), \
#                ('meat-veal', 'http://www.foodsubs.com/MeatVealLeg.html'), \
#                ('meat-veal', 'http://www.foodsubs.com/MeatVealBreast.html'), \
#                ('meat-veal', 'http://www.foodsubs.com/MeatVealMisc.html'), \
#                ('meat-other', 'http://www.foodsubs.com/MeatOther.html'), \
#                ('meat-other', 'http://www.foodsubs.com/Game.html'), \
#                ('meat-bacon', 'http://www.foodsubs.com/MeatcureBacon.html'), \
#                ('meat-sausage', 'http://www.foodsubs.com/MeatcureSausage.html'), \
#                ('meat-coldcuts', 'http://www.foodsubs.com/MeatcureCC.html'), \
#                ('meat-coldcuts', 'http://www.foodsubs.com/MeatcureHams.html'), \
#                ('meat-miscellaneous', 'http://www.foodsubs.com/MeatvarHeart.html'), \
#                ('meat-miscellaneous', 'http://www.foodsubs.com/MeatvarKidneys.html'), \
#                ('meat-miscellaneous', 'http://www.foodsubs.com/MeatvarLiver.html'), \
#                ('meat-miscellaneous', 'http://www.foodsubs.com/MeatvarMisc.html'), \
#                ('meat-miscellaneous', 'http://www.foodsubs.com/MeatDried.html'), \
#                ('seafood-fish', 'http://www.foodsubs.com/Ffirmlea.html'), \
#                ('seafood-fish', 'http://www.foodsubs.com/Ffirmfat.html'), \
#                ('seafood-fish', 'http://www.foodsubs.com/FflakleaCod.html'), \
#                ('seafood-fish', 'http://www.foodsubs.com/FflakleaDor.html'), \
#                ('seafood-fish', 'http://www.foodsubs.com/FflakleaDrum.html'), \
#                ('seafood-fish', 'http://www.foodsubs.com/FflakleaFlat.html'), \
#                ('seafood-fish', 'http://www.foodsubs.com/FflakleaMull.html'), \
#                ('seafood-fish', 'http://www.foodsubs.com/FflakleaPerch.html'), \
#                ('seafood-fish', 'http://www.foodsubs.com/FflakleaPike.html'), \
#                ('seafood-fish', 'http://www.foodsubs.com/FflakleaPorg.html'), \
#                ('seafood-fish', 'http://www.foodsubs.com/FflakleaRock.html'), \
#                ('seafood-fish', 'http://www.foodsubs.com/FflakleaTile.html'), \
#                ('seafood-fish', 'http://www.foodsubs.com/Fflakfat.html'), \
#                ('seafood-fish', 'http://www.foodsubs.com/Fishsmok.html'), \
#                ('seafood-shellfish', 'http://www.foodsubs.com/Shelfish.html'), \
#                ('seafood-shellfish', 'http://www.foodsubs.com/Shelfishcrab.html'), \
#                ('seafood-caviar', 'http://www.foodsubs.com/Caviar.html'), \
#                ('dairy-butters', 'http://www.foodsubs.com/Fatsoils.html'), \
#                ('oils', 'http://www.foodsubs.com/Oils.html'), \
#                ('flowers', 'http://www.foodsubs.com/Flowers.html'), \
#                ('miscellaneous', 'http://www.foodsubs.com/Misc.html')]

In [ ]:
#Scrape data and write to a csv (used above)

# fd = food_dictionary()
# for tup in scrape_list:
#     fd.turn_to_df(tup[0], tup[1])
# final_df = fd.df
# final_df.to_csv('final_recipe_list_categorized.csv', encoding='utf-8')